In [4]:
import numpy as np
import matplotlib.pyplot as plt

### Import data
copy the data file from postProcessing/forces/0/coefficient.data to ./data folder

In [6]:
filename = "plot_data/coefficient.dat"

#### Loading only Cd and Cl

In [7]:
data = np.loadtxt(filename, unpack=True, usecols=[0, 1, 3])
data = data[:, 6000:]

In [8]:
Data = np.column_stack((data[0], data[1], data[2]))
np.savetxt('plot_data/case/filtered_data.dat', Data, fmt='%.6e', delimiter='   ', newline='\n', header='Timestep     Cd             Cl\n', footer='')

In [9]:
mean_cd = np.mean(data[1])
std_cd = np.std(data[1])

mean_cl = np.mean(data[2])
std_cl = np.std(data[2])

#### Writing mean and std values in text file 
File will be saved in /plots dir

In [10]:
text_file = open("plot_data/case/mean_std.txt", "w")
text_file.write("Mean Value(Cd)         : %s\n" % mean_cd)
text_file.write("Standard deviation(Cd) : %s" % std_cd)
text_file.write("\n\n")
text_file.write("Mean Value(Cl)         : %s\n" % mean_cl)
text_file.write("Standard deviation(Cl) : %s" % std_cl)
text_file.close()

#### Ploting values of Cd and Cl after 3s 
File will be saved in /plots dir

In [11]:
plt.plot(data[0], data[1], linewidth=2)
plt.xlabel('time(s)')
plt.ylabel('Cd')
plt.title('Values of Cd after 3 s')
plt.savefig('plot_data/case/Cd.png')
plt.close()
plt.plot(data[0], data[2], linewidth=2)
plt.xlabel('time(s)')
plt.ylabel('Cd')
plt.title('Values of Cd after 3 s')
plt.savefig('plot_data/case/Cl.png')
plt.close()

### Ploting statastics, mean and standard deviation of Cd and Cl

In [13]:
m_data = np.genfromtxt('mesh_study/mesh_study.csv', delimiter=',')

In [14]:
plt.plot(m_data[:, 1], m_data[:, 2], linewidth=2)
plt.xlabel('mesh size in X direction')
plt.ylabel('mean value of Cd')
plt.title('Mesh size and Mean value of Cd after 3s')
plt.savefig('mesh_study/mesh_mean_cd.png')
plt.close()

plt.plot(m_data[:, 1], m_data[:, 3], linewidth=2)
plt.xlabel('mesh size in X direction')
plt.ylabel('standard deviation of Cd')
plt.title('Mesh size and standard deviation of Cd after 3s')
plt.savefig('mesh_study/mesh_std_cd.png')
plt.close()

plt.plot(m_data[:, 1], m_data[:, 4], linewidth=2)
plt.xlabel('mesh size in X direction')
plt.ylabel('mean value of Cl')
plt.title('Mesh size and Mean value of Cl after 3s')
plt.savefig('mesh_study/mesh_mean_cl.png')
plt.close()

plt.plot(m_data[:, 1], m_data[:, 5], linewidth=2)
plt.xlabel('mesh size in X direction')
plt.ylabel('standard deviation of Cl')
plt.title('Mesh size and standard deviation of Cl after 3s')
plt.savefig('mesh_study/mesh_std_cl.png')
plt.close()